In [4]:
!pip install pyarrow

     ---------------------------------------- 21.5/21.5 MB 2.2 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
yellow_jan = pd.read_parquet('data/yellow_tripdata_2022-01.parquet')
yellow_feb = pd.read_parquet('data/yellow_tripdata_2022-02.parquet')

##### Question 1:

In [7]:
# Question 1 = 19
len(yellow_jan.columns)

19

##### Question 2:

In [8]:
# Question 2:
yellow_jan.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0


In [9]:
yellow_jan['duration'] = yellow_jan.tpep_dropoff_datetime - yellow_jan.tpep_pickup_datetime
yellow_jan.duration = yellow_jan.duration.apply(lambda td: td.total_seconds() / 60)

In [10]:
yellow_jan.duration.std()

# Q2 response = 46.45

46.44530513776499

##### Question 3:

In [11]:
yellow_jan_frac = yellow_jan[(yellow_jan.duration >= 1) & (yellow_jan.duration <= 60)]

In [12]:
len(yellow_jan_frac) / len(yellow_jan)

# Question 3= 98%

0.9827547930522406

##### Question 4:

In [13]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

yellow_jan_frac[categorical] = yellow_jan_frac[categorical].astype(str)

C:\Users\crocs\AppData\Local\Temp\ipykernel_6012\831039043.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yellow_jan_frac[categorical] = yellow_jan_frac[categorical].astype(str)


In [14]:

train_dicts = yellow_jan_frac[categorical + numerical].to_dict(orient='records')


In [15]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = yellow_jan_frac[target].values

In [16]:
X_train_df = pd.DataFrame.sparse.from_spmatrix(X_train)

In [17]:
X_train_df.shape


# Question 4 = 515 columns

(2421440, 516)

##### Question 5:

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.001496179444567

In [19]:
#Question 5 answer = 6.99

##### Question 6:

In [20]:
yellow_feb['duration'] = yellow_feb.tpep_dropoff_datetime - yellow_feb.tpep_pickup_datetime

In [21]:
yellow_feb['duration'] = yellow_feb['duration'].dt.total_seconds()

In [22]:
yellow_feb['duration']  = yellow_feb['duration'] / 60

In [23]:
# subsetting

yellow_feb_frac = yellow_feb[(yellow_feb.duration >= 1) & (yellow_feb.duration <= 60)]

In [24]:
yellow_feb_frac

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.00,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.00,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.50,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.00,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.50,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979426,2,2022-02-28 23:50:00,2022-03-01 00:06:00,NaN,3.40,NaN,None,163,193,0,14.37,0.00,0.5,0.00,0.00,0.3,17.67,NaN,NaN,16.000000
2979427,2,2022-02-28 23:06:57,2022-02-28 23:19:12,NaN,3.48,NaN,None,141,4,0,14.51,0.00,0.5,2.00,0.00,0.3,19.81,NaN,NaN,12.250000
2979428,2,2022-02-28 23:48:13,2022-03-01 00:03:33,NaN,3.05,NaN,None,161,151,0,14.38,0.00,0.5,3.81,0.00,0.3,21.49,NaN,NaN,15.333333
2979429,2,2022-02-28 23:56:41,2022-03-01 00:04:57,NaN,2.62,NaN,None,141,226,0,12.53,0.00,0.5,1.71,0.00,0.3,17.54,NaN,NaN,8.266667


In [25]:
yellow_feb_frac[categorical] = yellow_feb_frac[categorical].astype(str)

C:\Users\crocs\AppData\Local\Temp\ipykernel_6012\1190947295.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yellow_feb_frac[categorical] = yellow_feb_frac[categorical].astype(str)


In [26]:
train_val_dicts = yellow_feb_frac[categorical + numerical].to_dict(orient='records')


In [27]:
X_val_train = dv.transform(train_val_dicts)

target = 'duration'
y_val_train = yellow_feb_frac[target].values

In [28]:
y_val_pred = lr.predict(X_val_train)

mean_squared_error(y_val_train, y_val_pred, squared=False)

7.795498792240916

In [ ]:
# Question 6 answer = 7.79